In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn 
import numpy as np
from itertools import product
import torch.nn 
from torch.optim import Adam, LBFGS, SGD
import re
import heapq as hq

In [ ]:
# your path to the data
dataset = pd.read_csv("/Users/summer/Desktop/CPSC532Project/novozymes-enzyme-stability-prediction/train.csv")
dataset

In [ ]:
# do not need seq_id and data_source
dataset.drop('seq_id',inplace= True, axis = 1)
dataset.drop('data_source',inplace= True, axis = 1)

In [ ]:
# unigram and bigram
Aminoacids = ['A', 'R', 'N','D','C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
Blocks = []

maxSize = 3
for k in range(1,maxSize):    
    Blocks.append([''.join(i) for i in product(Aminoacids, repeat = k)])


for i in Aminoacids:
    dataset[i] = dataset['protein_sequence'].str.count(i, re.I)
    
for i in Blocks[1]:
    dataset[i] = dataset['protein_sequence'].str.count(i, re.I)

In [ ]:
# take a look at dataset statas
dataset.describe()

In [ ]:
# only take data with reasonable features
dataset = dataset.loc[dataset["pH"] <= 14]

In [ ]:
# shuffle the data and make it training set and testing set
shuffled_dataset = dataset.sample(frac = 1)
cutoff = int(0.8 * len(shuffled_dataset))
training_set = shuffled_dataset.iloc[:cutoff]
testing_set = shuffled_dataset.iloc[cutoff:]

In [ ]:
training_set

In [ ]:
testing_set

In [ ]:
training_labels = training_set['tm']
testing_labels = testing_set['tm']

In [ ]:
testing_set.drop("tm", inplace = True, axis = 1)
training_set.drop("tm", inplace = True, axis = 1)

In [ ]:
testing_set

In [ ]:
testing_labels = torch.tensor(testing_labels.values.astype(np.float32))
training_labels = torch.tensor(training_labels.values.astype(np.float32))

In [ ]:
training_set.drop("protein_sequence", inplace = True, axis = 1)
testing_set.drop("protein_sequence", inplace = True, axis = 1)

In [ ]:
x_data = torch.tensor(training_set.values.astype(np.float32))
test_data = torch.tensor(testing_set.values.astype(np.float32))

# Linear regression

In [ ]:
y_data = training_labels

x = x_data
y = y_data

In [ ]:
X = torch.cat((x, torch.ones(x.size()[0],1)), 1)

A = torch.inverse(torch.mm(X.t(), X))
W = torch.mv(torch.mm(A, X.t()), y)

print(X)
print(W)

In [ ]:
test_w_bias = torch.cat((test_data, torch.ones(test_data.size()[0],1)), 1)


In [ ]:
yhat = torch.matmul(test_w_bias, W)
print(yhat)
print(testing_labels)
print(sum((yhat-testing_labels)**2)/len(yhat))
print(((yhat-testing_labels)**2))

### Linear regression with L1 reg

In [ ]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.1)
reg.fit(X, y_data)

reg.predict(test_w_bias)

In [ ]:
def KNN(training, testing, measure, *args):
    dictionary = []
    for j  in range(len(testing)):
        h = []
        print(j)
        for i in range(len(training)):
            distance = measure(testing[i], training[i])
            if len(h) == 0:
                heapq.heappush(h, (distance, i))
            if distance < h[0][0]:
                if len(h) > args[0]:
                    heapq.heappop(h)
                    heapq.heappush(h,(distance, i))
                else:
                    heapq.heappush(h,(distance, i))
        dictionary[j] = h
    return dictionary

In [ ]:
def min_distance( word1, word2) -> int:
    n, m = len(word1), len(word2)

    dp = [[0] * (m + 1) for _ in range(n + 1)]

    for i in range (n+1):
        dp[i][0] = i

    for i in range(m+1):
        dp[0][i] = i

    for i in range (0, n ):
        for j in range (0, m ):
            if word1[i] == word2[j]:
                dp[i+1][j+1] = dp[i][j]
            else:
                dp[i+1][j+1] = min(dp[i][j] + 1, dp[i][j+1] + 1, dp[i+1][j] + 1)


    return dp[-1][-1]

In [ ]:
# Linear regression/ with l1 reg / with l2 reg/ l1 l2 reg
# sklearn pytorch 

In [ ]:
# KNN (k value/ (elbow's methods), sample rate, stochastic KNN = sampling?)